# Unified View Creation
**Author:** Marang Mutloatse

**Description:** This script merges the prepared lab, patient and pharmacy datasets.
- *Pharmacy table* - `df_pharm`
- *Patient table* - `df_patient`
- *Laboratory table* - `df_labs`
- *Status history table* - `df_status`
- *Clinic table* - `df_clinic`
- *EAC table* - `df_eac`
- *OTZ table* - `df_otz`

**Version:** 0.0.1

**Status:** Development

## Import Libraries

In [5]:
import os
user_dev_path = os.path.dirname(os.getcwd())
# get parent path for package
package_path = os.path.dirname(user_dev_path)
import sys
import pandas as pd
import yaml
import datetime as dt
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
sys.path.insert(0,package_path + '/src/')
from hiv_support_package import visit_features,helper_functions
from sklearn.impute import SimpleImputer
from datetime import timedelta, date
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Loading data

In [6]:
try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [7]:
# Paths
input_root = config['rise_files']['raw_data_path']
# prepared datasets
patient_file = config['rise_files']['feature_patient_file']
lab_file = config['rise_files']['feature_lab_file']
pharmacy_file = config['rise_files']['feature_pharmacy_file']
status_file = config['rise_files']['raw_status_history_file']
clinic_file = config['rise_files']['feature_clinic_file']
eac_file = config['rise_files']['feature_eac_file']
otz_file = config['rise_files']['feature_otz_file']
patient_input = input_root + patient_file
lab_input = input_root + lab_file
pharmacy_input = input_root + pharmacy_file
status_input = input_root + status_file
clinic_input = input_root + clinic_file
eac_input = input_root + eac_file
otz_input = input_root + otz_file

print(patient_input)
print(lab_input)
print(pharmacy_input)
print(status_input)
print(clinic_input)
print(eac_input)
print(otz_input)

/data/rise_data/feature_patient.csv
/data/rise_data/feature_labs.csv
/data/rise_data/feature_pharmacy.csv
/data/rise_data/statushistory.csv
/data/rise_data/feature_clinic.csv
/data/rise_data/feature_eac.csv
/data/rise_data/feature_otz.csv


In [4]:
df_patient = pd.read_csv(patient_input)
df_labs = pd.read_csv(lab_input)
df_pharm = pd.read_csv(pharmacy_input)
df_clinic = pd.read_csv(clinic_input)
df_status = pd.read_csv(status_input,low_memory=False)
df_eac = pd.read_csv(eac_input)
df_otz = pd.read_csv(otz_input)

## Merge pharmacy

In [5]:
df_pharm = pd.merge(df_pharm,df_patient,left_on="V_PATIENT_ID",right_on="P_PATIENT_ID",how="left")
df_pharm = df_pharm[df_pharm['P_PATIENT_ID'].notnull()]

## Prep Status History Table

In [6]:
df_status = df_status.loc[:,['PATIENT_ID','CURRENT_STATUS','DATE_CURRENT_STATUS']]
df_status = df_status.add_prefix('S_')
df_status['S_DATE_CURRENT_STATUS'] = pd.to_datetime(df_status['S_DATE_CURRENT_STATUS'],format="%d/%m/%Y",errors="coerce")

In [7]:
visit_ids = df_pharm.V_PATIENT_ID.unique()
dedup_vp_count = len(df_pharm.index)
dedup_vp_count

606136

In [8]:
# Filter for labs id s in visit dataset
print("Before reducing size, shape: ",   visit_features.format_shape(df_status))
df_status = df_status.loc[df_status['S_PATIENT_ID'].isin(visit_ids)]
print("After reducing size, shape: ",   visit_features.format_shape(df_status))

Before reducing size, shape:  rows:602,341; cols: 3
After reducing size, shape:  rows:254,438; cols: 3


In [9]:
df_pharm.V_PATIENT_ID.nunique(),df_status.S_PATIENT_ID.nunique()

(78945, 75774)

In [10]:
df_status = df_status[df_status.S_DATE_CURRENT_STATUS.notnull()]
list_status_ids = df_status.S_PATIENT_ID.unique()
statusids = list_status_ids.tolist()
visitids = visit_ids.tolist()
diff_list = set(visitids) - set(statusids)
len(diff_list)

3176

In [11]:
df_status_sub = df_patient.loc[df_patient['P_PATIENT_ID'].isin(diff_list)]
df_status_sub = df_status_sub.loc[:,['P_PATIENT_ID','P_CURRENT_STATUS','P_DATE_CURRENT_STATUS']]
df_status_sub['P_DATE_CURRENT_STATUS'] = pd.to_datetime(df_status_sub['P_DATE_CURRENT_STATUS'])
df_status_sub['P_DATE_CURRENT_STATUS'] = df_status_sub['P_DATE_CURRENT_STATUS'].fillna(pd.to_datetime("2020-01-01 00:00:00"))
df_status_sub = df_status_sub.rename(columns = {'P_PATIENT_ID':'S_PATIENT_ID','P_DATE_CURRENT_STATUS':'S_DATE_CURRENT_STATUS','P_CURRENT_STATUS':'S_CURRENT_STATUS'})
df_status_merged = pd.concat([df_status_sub,df_status])

# Unified View Creation

## Merge Patient onto Pharmacy Visits Table

In [12]:
visit_cols = ['V_DATE_VISIT']
date_cols = ['L_DATE_REPORTED']
check_cols = ['V_DATE_VISIT','L_DATE_REPORTED']
clinic_cols = ['C_DATE_VISIT']
status_cols = ['S_DATE_CURRENT_STATUS']
eac_cols = ['E_EF_FIRST_DATE_EAC','E_DATE']
otz_cols = ['O_EF_FIRST_DATE_OTZ','O_DATE']

In [13]:
df_pharm = helper_functions.optimize(df_pharm,visit_cols)
df_labs = helper_functions.optimize(df_labs,date_cols)
df_status_merged = helper_functions.optimize(df_status_merged,status_cols)
df_clinic = helper_functions.optimize(df_clinic,clinic_cols)
df_eac = helper_functions.optimize(df_eac,eac_cols)
df_otz = helper_functions.optimize(df_otz,otz_cols)

In [14]:
df_pharm = df_pharm.sort_values(by=['V_PATIENT_ID','V_DATE_VISIT'])
df_labs = df_labs.sort_values(by=['L_PATIENT_ID','L_DATE_REPORTED'])

In [15]:
# Filter for labs id s in visit dataset
print("Before reducing size, shape: ",   visit_features.format_shape(df_labs))
df_labs = df_labs.loc[df_labs['L_PATIENT_ID'].isin(visit_ids)]
print("After reducing size, shape: ",   visit_features.format_shape(df_labs))

Before reducing size, shape:  rows:201,478; cols: 30
After reducing size, shape:  rows:125,169; cols: 30


## Merge 1st Unified view: Labs  + Pharmacy + Clinic

In [16]:
df_pharm = df_pharm.sort_values(by=['V_DATE_VISIT'])
df_labs = df_labs.sort_values(by=['L_DATE_REPORTED'])

In [17]:
df_uv = pd.merge_asof(df_pharm, 
                  df_labs, 
                  left_on="V_DATE_VISIT", 
                  right_on="L_DATE_REPORTED", 
                  left_by="V_PATIENT_ID", 
                  right_by="L_PATIENT_ID")

In [18]:
print("\n\n**** After Merge ****")
print("All: "+   visit_features.format_shape(df_uv))
print("L_MERGED_ID.notnull(): "+   visit_features.format_shape(df_uv[df_uv.L_PATIENT_ID.notnull()]))
print("V_MERGED_ID.notnull(): "+   visit_features.format_shape(df_uv[df_uv.V_PATIENT_ID.notnull()]))

match_ratio = len(df_uv[df_uv.V_PATIENT_ID.notnull()])/(len(df_pharm.index))
print("\n")
print("Match Ratio: "+ str(match_ratio))



**** After Merge ****
All: rows:606,136; cols: 231
L_MERGED_ID.notnull(): rows:254,350; cols: 231
V_MERGED_ID.notnull(): rows:606,136; cols: 231


Match Ratio: 1.0


## Merge 2nd Unified View: UV + Clinic

In [19]:
df_clinic = df_clinic.sort_values(by=['C_PATIENT_ID','C_DATE_VISIT'])

In [20]:
# Filter for labs id s in visit dataset
print("Before reducing size, shape: ",   visit_features.format_shape(df_clinic))
df_clinic = df_clinic.loc[df_clinic['C_PATIENT_ID'].isin(visit_ids)]
print("After reducing size, shape: ",   visit_features.format_shape(df_clinic))

Before reducing size, shape:  rows:802,224; cols: 85
After reducing size, shape:  rows:453,112; cols: 85


In [21]:
df_uv = df_uv.sort_values(by=['V_DATE_VISIT'])
df_clinic = df_clinic.sort_values(by=['C_DATE_VISIT'])

In [22]:
df_uv = pd.merge_asof(df_uv, 
                  df_clinic, 
                  left_on="V_DATE_VISIT", 
                  right_on="C_DATE_VISIT", 
                  left_by="V_PATIENT_ID", 
                  right_by="C_PATIENT_ID")

## Merge 3rd Unified View: UV + Status History

In [23]:
df_uv = df_uv.sort_values(by=['V_DATE_VISIT'])
df_status_merged['S_PATIENT_ID'] = df_status_merged['S_PATIENT_ID'].astype('int32') 
df_status_merged = df_status_merged.sort_values(by=['S_DATE_CURRENT_STATUS'])

In [24]:
df_uv = pd.merge_asof(df_uv, 
                  df_status_merged, 
                  left_on="V_DATE_VISIT", 
                  right_on="S_DATE_CURRENT_STATUS", 
                  left_by="V_PATIENT_ID", 
                  right_by="S_PATIENT_ID")

In [25]:
df_uv['S_CURRENT_STATUS'] = df_uv['S_CURRENT_STATUS'].str.title()
df_uv = df_uv.sort_values(by=['V_PATIENT_ID','V_DATE_VISIT'])
m3 = df_uv.groupby('V_PATIENT_ID')['S_CURRENT_STATUS'].shift().ne(df_uv['S_CURRENT_STATUS'])
m4 = df_uv['S_CURRENT_STATUS'].isin(["Art Restart", "Art Transfer In","Lost To Follow Up"])
df_uv['STATUS_CHANGE'] = (m3 & m4).astype(int)

## Merge 4rd Unified View: UV + EAC

In [26]:
# Filter for labs id s in visit dataset
print("Before reducing size, shape: ",   visit_features.format_shape(df_eac))
df_eac = df_eac.loc[df_eac['E_PATIENT_ID'].isin(visit_ids)]
print("After reducing size, shape: ",   visit_features.format_shape(df_eac))

Before reducing size, shape:  rows:31,518; cols: 5
After reducing size, shape:  rows:21,183; cols: 5


In [27]:
df_eac['E_PATIENT_ID'] = df_eac['E_PATIENT_ID'].astype('int32')
df_uv['V_PATIENT_ID'] = df_uv['V_PATIENT_ID'].astype('int32')

In [28]:
df_uv = df_uv.sort_values(by=['V_DATE_VISIT'])
df_eac = df_eac.sort_values(by=['E_DATE'])

In [29]:
df_uv = pd.merge_asof(df_uv,
                  df_eac, 
                  left_on="V_DATE_VISIT", 
                  right_on="E_DATE", 
                  left_by="V_PATIENT_ID", 
                  right_by="E_PATIENT_ID")

In [30]:
df_uv = df_uv.sort_values(by=['V_PATIENT_ID','V_DATE_VISIT'])
df_uv['E_EAC_COUNT'] = df_uv['E_EAC_COUNT'].fillna(0.0)
df_uv['E_EF_MONTHS_SINCE_FIRST_EAC'] = df_uv['E_EF_MONTHS_SINCE_FIRST_EAC'].fillna(0.0)

# Get eac iniiation
df_uv['E_EF_EAC_STARTED'] = 0
df_uv.loc[df_uv['E_DATE'].isna().groupby(df_uv['V_PATIENT_ID']).idxmin(), 'E_EF_EAC_STARTED'] = 1

# Backfill missing first date
df_uv['E_EF_FIRST_DATE_EAC'] = df_uv.groupby('V_PATIENT_ID')['E_EF_FIRST_DATE_EAC'].transform(
    lambda x: x.bfill()
)

#Days between current arv pickup and counselling session
df_uv['E_DAYS_SINCE_LAST_EAC_AT_ARV_PICKUP'] = (df_uv["V_DATE_VISIT"] - df_uv["E_DATE"])/np.timedelta64(1,'D')
df_uv['E_MONTHS_SINCE_LAST_EAC_AT_ARV_PICKUP'] = (df_uv["V_DATE_VISIT"] - df_uv["E_DATE"])/np.timedelta64(1,'M')
df_uv['E_DAYS_SINCE_LAST_EAC_AT_ARV_PICKUP'] = df_uv['E_DAYS_SINCE_LAST_EAC_AT_ARV_PICKUP'].fillna(0)
df_uv['E_MONTHS_SINCE_LAST_EAC_AT_ARV_PICKUP'] = df_uv['E_MONTHS_SINCE_LAST_EAC_AT_ARV_PICKUP'].fillna(0)

## Merge 5th Unified View: UV + OTZ

In [31]:
# Filter for labs id s in visit dataset
print("Before reducing size, shape: ",   visit_features.format_shape(df_otz))
df_otz = df_otz.loc[df_otz['O_PATIENT_ID'].isin(visit_ids)]
print("After reducing size, shape: ",   visit_features.format_shape(df_otz))

Before reducing size, shape:  rows:7,327; cols: 5
After reducing size, shape:  rows:4,940; cols: 5


In [32]:
df_uv = df_uv.sort_values(by=['V_DATE_VISIT'])
df_otz = df_otz.sort_values(by=['O_DATE'])

In [33]:
df_uv = pd.merge_asof(df_uv, 
                  df_otz, 
                  left_on="V_DATE_VISIT", 
                  right_on="O_DATE", 
                  left_by="V_PATIENT_ID", 
                  right_by="O_PATIENT_ID")

In [34]:
df_uv = df_uv.sort_values(by=['V_PATIENT_ID','V_DATE_VISIT'])
df_uv['O_OTZ_COUNT'] = df_uv['O_OTZ_COUNT'].fillna(0.0)

# Get eac iniiation
df_uv['O_EF_EAC_STARTED'] = 0
df_uv.loc[df_uv['O_DATE'].isna().groupby(df_uv['V_PATIENT_ID']).idxmin(), 'E_EF_OTZ_STARTED'] = 1

# Backfill missing first date
df_uv['O_EF_FIRST_DATE_OTZ'] = df_uv.groupby('V_PATIENT_ID')['O_EF_FIRST_DATE_OTZ'].transform(
    lambda x: x.bfill()
)

#Days between current arv pickup and counselling session
df_uv['O_DAYS_SINCE_LAST_OTZ_AT_ARV_PICKUP'] = (df_uv["V_DATE_VISIT"] - df_uv["O_DATE"])/np.timedelta64(1,'D')
df_uv['O_MONTHS_SINCE_LAST_OTZ_AT_ARV_PICKUP'] = (df_uv["V_DATE_VISIT"] - df_uv["O_DATE"])/np.timedelta64(1,'M')
df_uv['O_DAYS_SINCE_LAST_OTZ_AT_ARV_PICKUP'] = df_uv['O_DAYS_SINCE_LAST_OTZ_AT_ARV_PICKUP'].fillna(0)
df_uv['O_MONTHS_SINCE_LAST_OTZ_AT_ARV_PICKUP'] = df_uv['O_MONTHS_SINCE_LAST_OTZ_AT_ARV_PICKUP'].fillna(0)

# Post UV Creation Feature Engineering

In [35]:
df_uv["P_DATE_BIRTH"] = df_uv["P_DATE_BIRTH"].astype(str)
df_uv["P_DATE_REGISTRATION"] = df_uv["P_DATE_REGISTRATION"].astype(str)
df_uv["V_EF_FIRST_DATE_VISIT"] = df_uv["V_EF_FIRST_DATE_VISIT"].astype(str)
df_uv["P_DATE_STARTED"] = df_uv["P_DATE_STARTED"].astype(str)
df_uv["P_DATE_REGISTRATION"] = pd.to_datetime(df_uv["P_DATE_REGISTRATION"], errors='coerce')
df_uv["V_EF_FIRST_DATE_VISIT"] = pd.to_datetime(df_uv["V_EF_FIRST_DATE_VISIT"], errors='coerce')
df_uv["P_DATE_STARTED"] = pd.to_datetime(df_uv["P_DATE_STARTED"], errors='coerce')
df_uv["P_DATE_BIRTH"] = pd.to_datetime(df_uv["P_DATE_BIRTH"], errors='coerce')

In [36]:
df_uv = df_uv.assign(EF_START_DATE_TREATMENT = df_uv.filter(["P_DATE_REGISTRATION","V_EF_FIRST_DATE_VISIT","P_DATE_STARTED"])
                                .bfill(axis=1)
                                .ffill(axis=1)
                                .min(axis=1))

In [37]:
df_uv["EF_START_DATE_TREATMENT"] = pd.to_datetime(df_uv["EF_START_DATE_TREATMENT"],errors='coerce')
df_uv['EF_AGE_AT_START'] = (df_uv["EF_START_DATE_TREATMENT"] - df_uv["P_DATE_BIRTH"])/np.timedelta64(1,'Y')
df_uv['EF_AGE_AT_VISIT'] = (df_uv["V_DATE_VISIT"] - df_uv["P_DATE_BIRTH"])/np.timedelta64(1,'Y')
df_uv.loc[df_uv['EF_AGE_AT_VISIT'] < 0,'EF_AGE_AT_VISIT'] = 0
df_uv.loc[df_uv['EF_AGE_AT_START'] < 0,'EF_AGE_AT_START'] = 0
df_uv['EF_TIME_IN_TREATMENT_MONTHS'] = (df_uv["V_DATE_VISIT"] - df_uv["EF_START_DATE_TREATMENT"])/np.timedelta64(1,'M')

## Push to VM

In [38]:
#drops_id_cols = ["V_PHARMACY_ID","L_LABORATORY_ID","P_REGIMENTYPE"]
df_uv.to_csv(input_root+'unified_view.csv',index=False,header=True)